In [1]:
# --- 1. SETUP ---
import os
os.system('pip install sentence-transformers wordcloud')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
from wordcloud import WordCloud
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA

sns.set_theme(style="whitegrid")
plt.rcParams['figure.figsize'] = (14, 8)
if not os.path.exists('images'): os.makedirs('images')

# --- 2. LOAD DATA FOR VISUALIZATION (The "Good" Dataset) ---
# We use perfumes_cleaned.csv for charts because it has the clean Brand names you liked.
file_visuals = "perfumes_cleaned.csv"
file_ratings = "fra_perfumes.csv"

if not os.path.exists(file_visuals) or not os.path.exists(file_ratings):
    print("❌ ERROR: Please upload BOTH 'perfumes_cleaned.csv' and 'fra_perfumes.csv'")
else:
    df = pd.read_csv(file_visuals)
    print(f"✅ Loaded Visual Data: {len(df)} rows")

    # --- 3. CLEANING (RESTORING ORIGINAL LOGIC) ---
    # This cleaning logic replicates your original notebook to match the old charts
    def clean_text(text):
        if pd.isna(text): return ""
        # Remove brackets and quotes: ['Note'] -> Note
        text = re.sub(r"[\[\]'\"/]", "", str(text))
        return text.strip()

    df['Notes_Clean'] = df['Notes'].apply(clean_text)

    # Ensure Brand is string
    df['Brand'] = df['Brand'].astype(str)

    # Remove empty rows
    df.dropna(subset=['Notes_Clean', 'Name'], inplace=True)
    df['word_count'] = df['Description'].astype(str).apply(lambda x: len(x.split()))

    print("Data Restored to Original Format.")

    # --- 4. GENERATING CHARTS ---
    print("\n--- Generating Charts (Original Style) ---")

    # CHART 1: TOP BRANDS (Restored)
    # This will show "TOM FORD Private Blend" as #1 again
    plt.figure(figsize=(12, 6))
    top_b = df['Brand'].value_counts().head(20)
    sns.barplot(x=top_b.values, y=top_b.index, hue=top_b.index, legend=False, palette='viridis')
    plt.title('Top 20 Most Frequent Brands')
    plt.savefig('images/top_brands_frequency.png', bbox_inches='tight')
    plt.close()
    print("📊 1. Top Brands Saved (Matched Original).")

    # CHART 2: TOP NOTES (Restored)
    all_notes = ",".join(df['Notes_Clean'].tolist())
    notes_list = [n.strip() for n in all_notes.split(',')]
    top_n = pd.Series(notes_list).value_counts().head(20)

    plt.figure(figsize=(12, 6))
    sns.barplot(x=top_n.values, y=top_n.index, hue=top_n.index, legend=False, palette='magma')
    plt.title('Top 20 Most Common Scent Notes')
    plt.savefig('images/top_notes_frequency2.png', bbox_inches='tight')
    plt.close()
    print("📊 2. Top Notes Saved (Matched Original).")

    # CHART 3: WORD CLOUD (Restored)
    wordcloud = WordCloud(width=800, height=400, background_color='white', colormap='viridis').generate(all_notes)
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title("Olfactory DNA")
    plt.savefig('images/cloud_perfume.png', bbox_inches='tight')
    plt.close()
    print("📊 3. Word Cloud Saved (Matched Original).")

    # CHART 4: CLUSTERS (THE ONLY FIX)
    # We recreate the clusters but ADD NAMES
    print("   Running Clustering for Chart 4...")
    tfidf = TfidfVectorizer(stop_words='english', max_features=1000)
    matrix = tfidf.fit_transform(df['Notes_Clean'])

    # 8 Clusters (Same as original)
    kmeans = KMeans(n_clusters=8, random_state=42, n_init=10)
    clusters = kmeans.fit_predict(matrix)

    # Smart Naming Function
    def get_labels(data, cls, vec):
        df_tmp = pd.DataFrame(data.todense()).groupby(cls).mean()
        terms = vec.get_feature_names_out()
        labels = {}
        for i, r in df_tmp.iterrows():
            top = r.argsort()[-2:][::-1]
            labels[i] = " & ".join([terms[t].title() for t in top])
        return labels

    df['Cluster Name'] = pd.Series(clusters).map(get_labels(matrix, clusters, tfidf))

    plt.figure(figsize=(14, 8))
    sns.violinplot(x='Cluster Name', y='word_count', hue='Cluster Name', legend=False, data=df, palette="Spectral")
    plt.title("Description Length vs. Scent Family")
    plt.xticks(rotation=45, ha='right')
    plt.xlabel("Scent Family (Cluster)")
    plt.ylabel("Word Count")
    plt.savefig('images/scent_clusters_pca.png', bbox_inches='tight')
    plt.close()
    print("📊 4. Cluster Chart Saved (Fixed Labels).")

    # --- 5. MERGE & SAVE FOR APP ---
    print("\n--- Creating Final App Files ---")

    # Now we bring in Ratings/Gender from fra_perfumes for the App functionality
    df_fra = pd.read_csv(file_ratings)

    # Keys
    df['key'] = df['Name'].astype(str).str.lower().str.strip()
    df_fra['key'] = df_fra['Name'].astype(str).str.lower().str.strip()

    # Rename FRA cols
    cols_fra = [c.lower() for c in df_fra.columns]
    rename_fra = {}
    if 'rating value' in cols_fra: rename_fra[df_fra.columns[cols_fra.index('rating value')]] = 'Score'
    if 'gender' in cols_fra: rename_fra[df_fra.columns[cols_fra.index('gender')]] = 'Gender'
    df_fra.rename(columns=rename_fra, inplace=True)

    # Merge
    cols_to_pull = ['key']
    if 'Score' in df_fra.columns: cols_to_pull.append('Score')
    if 'Gender' in df_fra.columns: cols_to_pull.append('Gender')

    df_final = pd.merge(df, df_fra[cols_to_pull], on='key', how='left')
    df_final['Score'] = df_final['Score'].fillna(0.0)
    df_final['Gender'] = df_final['Gender'].fillna('Unisex')

    # --- 6. SAVE AI MODEL ---
    # Optimization: Top 5000
    if len(df_final) > 5000: df_model = df_final.head(5000).copy()
    else: df_model = df_final.copy()

    model = SentenceTransformer('all-MiniLM-L6-v2')
    # Use Description if available, else Notes
    df_model['Description'] = df_model['Description'].fillna(df_model['Notes_Clean'])
    embeddings = model.encode(df_model['Description'].tolist(), show_progress_bar=True)

    matrix_trim = tfidf.fit_transform(df_model['Notes_Clean'])

    sim_desc = cosine_similarity(embeddings, embeddings).astype(np.float32)
    sim_notes = cosine_similarity(matrix_trim, matrix_trim).astype(np.float32)
    hybrid = (0.7 * sim_desc) + (0.3 * sim_notes)

    np.save('hybrid_similarity.npy', hybrid)
    df_model.rename(columns={'Notes_Clean': 'Notes'}, inplace=True)
    df_model.to_csv('perfumes_dataset.csv', index=False)

    print("\n🎉 DONE! Download files (csv, npy, images).")

✅ Loaded Visual Data: 2191 rows
Data Restored to Original Format.

--- Generating Charts (Original Style) ---
📊 1. Top Brands Saved (Matched Original).
📊 2. Top Notes Saved (Matched Original).
📊 3. Word Cloud Saved (Matched Original).
   Running Clustering for Chart 4...
📊 4. Cluster Chart Saved (Fixed Labels).

--- Creating Final App Files ---


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/69 [00:00<?, ?it/s]


🎉 DONE! Download files (csv, npy, images).
